# Иcточники данных

- Внешние источники
  - b1: `b1_analytics.pkl`
  - kamaflow: `kamaflow_researches.pkl`
- Внутренние источники
  - Отраслевой хаб: данные выложены в sberdisk

## Формат данных 

Результат представляет собой массив словарей, где каждый словарь содержит информацию об одном аналитическом материале. Каждый словарь имеет следующие ключи:

### Пример элемента списка `analytics`:
```json
{
    "link": "/news/12345",
    "date": "2023-10-15",
    "name": "Анализ рынка ИТ в 2023 году",
    "name_en": "it-market-analysis-2023",
    "tags": ["ИТ", "аналитика", "2023"],
    "pdf_links": [
        "https://b1.ru/local/assets/surveys/it-market-analysis-2023.pdf"
    ],
    "text": "В 2023 году рынок ИТ показал рост на 14%...",
    "pdfs": [
        {
            "name": "it-market-analysis-2023.pdf",
            "content": "binary_pdf_content_here"
        }
    ]
}
```
Мы хотим получить такой, при этом часть полей можент быть пустой:
```json
{
    "link": "/news/12345",
    "date": "2023-10-15",
    "name": "Анализ рынка ИТ в 2023 году",
    "name_en": "it-market-analysis-2023",
    "tags": ["ИТ", "аналитика", "2023"],
    "sber_tags": ["Теги", "из", "сбера", "отрасль", "из", "kksb_clinet_profile"],
    "pdf_links": [
        "https://b1.ru/local/assets/surveys/it-market-analysis-2023.pdf"
    ],
    "text": "В 2023 году рынок ИТ показал рост на 14%...",
    "summary": "Саммари на основе pdfs",
    "pdfs": [
        {
            "name": "it-market-analysis-2023.pdf",
            "content": "binary_pdf_content_here",
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        }
    ]
}
```

Для дальнейнего анализа нужно только следующие поля: date, sber_tags, summary, pdfs['full_text'], pdfs['summary']:
```json
{
    "date": "2023-10-15",
    "sber_tags": ["Теги", "из", "сбера", "отрасль", "из", "kksb_clinet_profile"],
    "summary": "Саммари",
    "pdfs": [
        {
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        },
        {
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        }
    ]
}
```

In [1]:
import pickle


with open('b1_analytics.pkl', 'rb') as f:
    b1_analytics = pickle.load(f)

with open('kamaflow_researches.pkl', 'rb') as f:
    kamaflow_researches = pickle.load(f)

# with open('/Users/22926900/Desktop/consult_plan_v3/industry_analytics_parser/notebooks/industry_hub_analytics_update.pkl', 'rb') as f:
#     industry_hub_analytics = pickle.load(f)

docs = b1_analytics + kamaflow_researches #+ industry_hub_analytics

In [3]:
import re
import json

def extract_json_blocks(text):
    """
    Извлекает содержимое всех блоков ```json ... ``` из текста.
    
    Аргументы:
        text (str): Исходный текст, содержащий блоки с JSON.
        
    Возвращает:
        str: Содержимое всех JSON-блоков, объединенное через пробел.
             Если блоков не найдено, возвращает пустую строку.
    """
    pattern = r'```json(.*?)```'
    matches = re.findall(pattern, text, re.DOTALL)
    if not matches:
        return ""
    
    # Объединяем все найденные блоки через пробел и убираем лишние пробелы
    result = ' '.join(match.strip() for match in matches)
    return result

def fix_json_commas(json_str: str) -> str:
    """
    Fixes common JSON formatting issues related to commas.
    
    Args:
        json_str (str): Input JSON string
        
    Returns:
        str: Properly formatted JSON string
    """
    # Remove trailing commas before closing braces/brackets
    json_str = json_str.replace(",}", "}")
    json_str = json_str.replace(",\n}", "}")
    json_str = json_str.replace(",\n  }", "}")
    json_str = json_str.replace(",\n    }", "}")
    json_str = json_str.replace(",]", "]")
    json_str = json_str.replace(",\n]", "]")
    json_str = json_str.replace(",\n  ]", "]")
    json_str = json_str.replace(",\n    ]", "]")
    
    # Add missing commas between objects
    json_str = json_str.replace("}\n  {", "},\n  {")
    json_str = json_str.replace("}\n    {", "},\n    {")
    
    return json_str

## Добавляем саммари

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
import pandas as pd
import tempfile
from langchain_core.prompts import SystemMessagePromptTemplate
from tqdm import tqdm
import json


llm = ChatOpenAI(model_name="gpt-4.1-mini")


prompt = """
Ты стратегический консультант. Ты изучаешь и обрабатываешь отраслевые данные, новости, исследжования.
Твоя задача - провести анализ и сформировать структурированный отчёт о 1-3 ключевых трендах в формате JSON.

## JSON Schema
```json
{{
  "type": "array",
  "items": {{
    "type": "object",
    "required": ["тренд", "факт", "анализ"],
    "properties": {{
      "тренд": {{
        "type": "string",
        "description": "Краткая суть тренда в виде законченного утверждения. Например: 'рост спроса на экологичные товары'"
        }},
      "факт": {{
        "type": "string",
        "description": "Конкретные цифры, данные или утверждения из текста, подтверждающие тренд. Например: '73% потребителей готовы платить больше за экологичную упаковку'"
        }}, 
      "анализ": {{
        "type": "string",
        "description": "3-7 связанных предложений, раскрывающих: причины тренда, его последствия, влияние на рынок/бизнес, прогноз развития"
        }}
    }}
  }}
}}
```

## Правила валидации JSON:
1. Используй ТОЛЬКО двойные кавычки (") для строк
2. Ставь запятую после КАЖДОГО элемента, кроме последнего
3. НЕ СТАВЬ запятую после последнего элемента в объекте или массиве
4. Все скобки {{}} и [] должны быть закрыты правильно
5. Экранируй кавычки внутри строк как \\"
6. Не используй комментарии в JSON
7. При отсутствии данных возвращай пустой массив []
8. Экранируй спецсимволы согласно спецификации JSON:
   - \\" для кавычек
   - \\\\ для обратной косой черты
   - \\n для переноса строки
   - \\t для табуляции
8. JSON должен быть валидным для стандартного парсера

## Формат вывода:
```json
[
  {{
    "тренд": "суть тренда из данных",
    "факт": "подтверждающие факты",
    "анализ": "анализ тренда в 5-7 предложений"
  }}
]
```

## Пример правильного JSON без лишних запятых:
```json
[
  {{
    "тренд": "пример тренда",
    "факт": "пример факта",
    "анализ": "пример анализа"
  }},
  {{
    "тренд": "второй тренд",
    "факт": "второй факт",
    "анализ": "второй анализ"
  }}
]
```

## Требования к содержанию:
1. Извлекай только явные тренды из текста
2. Подтверждай трендыконкретными фактами
3. Анализируй тренд в 5-7 предложений
4. Пиши только на русском языке
5. Возвращай только валидный JSON
6. Все кавычки внутри строк обязательно экранируй: \\" — иначе JSON будет невалидным


## Входные данные:
- Название: {name}
- Теги: {tags} 
- Краткое содержание: {additional_text}
- Текст документа:
```
{text_document}
```
"""

prompt = """
Ты — стратегический консультант. На основе отраслевых данных, новостей и исследований тебе нужно выявить 1–3 ключевых тренда, подтверждённых фактами, и представить их в виде валидного JSON по заданной схеме.

## Твоя задача:
1. Найди до трёх трендов из текста.
2. Каждый тренд должен быть явно выражен, подтверждён конкретным фактом и проанализирован.
3. Анализ включает причины, последствия, влияние на рынок и прогноз развития (3–7 предложений).
4. Пиши только на русском языке.

## Формат вывода (JSON):
```json
[
  {{
    "тренд": "Краткое утверждение, отражающее суть тренда (например: \\"рост спроса на экологичные товары\\")",
    "факт": "Цифры, статистика или цитата из текста, подтверждающая тренд (например: \\"73% потребителей готовы платить больше за экологичную упаковку\\")",
    "анализ": "Связный аналитический текст из 3–7 предложений, раскрывающий причины, влияние, последствия и прогноз тренда"
  }}
]
```

## Требования к JSON:
- Используй только двойные кавычки (")
- После каждого поля ставь запятую, кроме последнего
- Не используй комментарии
- Все кавычки внутри строк экранируй: \\" 
- Экранируй спецсимволы: \\\\ — обратная косая, \\n — перенос строки, \\t — табуляция
- При отсутствии трендов верни: `[]`
- Все кавычки внутри строк обязательно экранируй: \\" — иначе JSON будет невалидным
- JSON должен быть полностью валидным

## Входные данные:
- Название: {name}
- Теги: {tags} 
- Краткое содержание: {additional_text}
- Текст документа:
```
{text_document}
```
"""

messages = ChatPromptTemplate.from_messages([
    ('system', prompt),
    ('human', "Выведи итоговый JSON"),
])


chain = (
    messages
    | llm 
    | (lambda x: fix_json_commas(x.content))
    | JsonOutputParser()
)

In [5]:
counter = 0
for doc in tqdm(docs):
    name = doc['name']
    tags = doc['tags']
    additional_text = doc['text']
    
    text_documents = []
    
    for pdf in doc['pdfs']:
        pdf_name = pdf['name']

        pdf_content = pdf.get('content', None)
        full_text = pdf.get('full_text', None)
        if full_text is None or len(full_text) > 0:
            counter += 1
        if full_text is None and pdf_content is not None:
            try:
                with tempfile.NamedTemporaryFile() as temp_file:
                    temp_file.write(pdf_content)
                    temp_file_path = temp_file.name
                    loader = PyPDFLoader(temp_file_path)
                    pdf_data = loader.load()
            except Exception as e:
                print(f"Error processing PDF {pdf_name}: {e}")
                pdf_data = None
            if pdf_data:
                full_text = "\n".join([doc.page_content for doc in pdf_data])
                pdf['full_text'] = full_text
        
        if full_text:
            if additional_text is None or len(additional_text) == 0:
                additional_text = 'нет краткого описание'
            text_document = f'**Документ с наименованием {pdf_name}, его содержимое:** {full_text}'
            text_documents.append(text_document)
    if len(text_documents) == 0:
        doc['summary'] = ''
    else:
        text_document = "\n".join(text_documents)
        doc['summary'] = chain.invoke(
                {
                    'name': name,
                    'tags': ", ".join(tags), 
                    'additional_text': additional_text,
                    'text_document': text_document[:50_000 - 3] + '...',
                }
            )

  0%|          | 0/99 [00:00<?, ?it/s]

 96%|█████████▌| 95/99 [18:58<00:55, 13.94s/it]invalid pdf header: b'RIFF\x8c'
EOF marker not found


Error processing PDF Супертренд на здоровое питание — ключевая тенденция на рынке хлебобулочных и кондитерских изделий в России: Stream has ended unexpectedly


100%|██████████| 99/99 [19:11<00:00, 11.63s/it]


## Добавляем теги
Теги из kksb_client_profile

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# Данные
data = {
    "date": "2023-10-15",
    "summary": "Саммари",
    "pdfs": [
        {
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        },
        {
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        }
    ]
}

# llm = ChatOpenAI(model_name="gpt-4o-mini")

# Возможные теги
tags_list = [
    'Энергетика', 'Лизинг', 'Нефтегазовая промышленность', 'Пищевая промышленность',
    'Финансы', 'Органы государственного и муниципального управления', 'Материалы', 'Промышленность',
    'Телекоммуникации', 'Телекоммуникации и медиа', 'ИТ-технологии', 'ЖКХ', 'Услуги',
    'Электроэнергетика и ЖКХ', 'Строительные подрядчики', 'Сельское хозяйство', 'Товары первой необходимости',
    'Энергоносители', 'Товары выборочного спроса', 'Операции с недвижимым имуществом', 'Государство',
    'Недвижимость', 'Фармацевтика и здравоохранение', 'Химическая промышленность', 'Розничная торговля товарами первой необходимости',
    'Машиностроение', 'Металлургическая и горнодобывающая промышленность', 'Производство строительных материалов',
    'Розничная торговля товарами выборочного спроса', 'Финансовая деятельность', 'Легкая промышленность',
    'Транспорт и логистика', 'Производство потребительских товаров', 'Лесная и деревообрабатывающая и целлюлозно-бумажная промышленность', 'Туризм'
]
tags_list = [tag.lower() for tag in tags_list]

for data in tqdm(docs):
    text_sources = [
        'Отраслевые теги из документа: ',
        ', '.join([x for x in data["tags"] if x != 'Прочее']), 
        '\n Текст трендов: ', str(data["summary"]), 
    ]
    context_text = "\n".join(filter(None, text_sources))[:40_000]
    prompt = PromptTemplate(
        input_variables=["text", "tags"],
        template=(
            """Определи, какие из следующих тегов отрасли наиболее подходят для данного текста с отраслевыми трендами:
            ## Теги отрасли: 
            {tags}. 
            ## Тренды:
            {text}
            ## Формат ответа:
            Отвечай только списком релевантных трендам тегов через запятую, без пояснений. Пиши только те теги, которые релевантны трендам и есть в списке теги отрасли:{tags}. Не выдумывай другие теги и не изменя  формулировку этих тегов. Не пиши ничего кроме тегов. Если релевантных тегов нет не пиши ничего."""
        )
    )

    response = llm.invoke(prompt.format(text=context_text, tags=", ".join(tags_list))).content.strip()
    data["sber_tags"] = [tag.strip() for tag in response.split(",") if tag.strip().lower() in tags_list]


100%|██████████| 99/99 [02:00<00:00,  1.22s/it]


### Пример

# Сохраняемся

In [7]:
with open('docs.pkl', 'wb') as f:
    pickle.dump(docs, f)

In [8]:
def parse_date(date_str):
    # Пробуем разные форматы дат
    for fmt in ('%d.%m.%Y', '%Y-%m-%d', '%d-%m-%Y', '%Y.%m.%d', '%y-%m-%d', '%d/%m/%Y'):
        try:
            return pd.to_datetime(date_str, format=fmt, errors='raise')
        except:
            continue
    return pd.NaT

In [ ]:
from datetime import datetime


new_docs = []
for row_counter, doc in enumerate(docs):
    new_doc = {
        "date": doc['date'],
        "sber_tags": ', '.join(list(set(doc['sber_tags']))),
        "summary": doc['summary'],
        'row_id': row_counter,
    }
    if isinstance(new_doc['date'], datetime):
        new_doc['date'] = new_doc['date'].strftime('%d.%m.%Y')
    # pdfs = []
    # for pdf in doc['pdfs']:
    #     pdfs.append({
    #         "full_text": pdf.get('full_text', ''),
    #         # "summary": pdf['summary'],
    #     })
    # new_doc['pdfs'] = pdfs
    new_docs.append(new_doc)

In [18]:
df = pd.DataFrame(new_docs).rename(columns={'date': 'date_sum'})
df['date_sum'] = df['date_sum'].apply(parse_date).dt.strftime('%d.%m.%Y')
df['sber_tags'] = df['sber_tags'].str.replace('\s*,\s*', ',', regex=True).str.split(',')
df = df.explode('sber_tags')
df['sber_tags'] = df['sber_tags'].str.strip()
df = df.reset_index(drop=True)
df = df.dropna(subset=['date_sum'], how='any')
df.to_excel('trends_with_dates.xlsx', index=False)